In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
schema = pd.read_sql_query("PRAGMA table_info('test')", conn)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [4]:
pd.read_sql_query("SELECT * FROM test LIMIT 10", conn)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [5]:
query = """
select uid, cast(MIN(julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24 as integer) as min_delta
from test t
join deadlines d on t.labname = d.labs
where t.labname != 'project1'
"""

pd.read_sql_query(query, conn)

,uid,min_delta
0,user_30,-202


In [14]:
query = """
select uid, cast(MAX(julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24 as integer) as max_delta
from test t
join deadlines d on t.labname = d.labs
where t.labname != 'project1'
"""

pd.read_sql_query(query, conn)

,uid,max_delta
0,user_25,-2


In [70]:
query = """
select avg(cast((julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24 as integer)) as avg_delta
from test t
join deadlines d on t.labname = d.labs
where t.labname != 'project1'
"""

pd.read_sql_query(query, conn)

,avg_delta
0,-89.125


In [71]:
query = """
select
    t.uid,
    avg(julianday(t.first_commit_ts) - julianday(d.deadlines, 'unixepoch')) * 24 as avg_diff,
    count(*) as pageviews
from test t
join deadlines d
    on t.labname = d.labs
where t.labname != 'project1'
group by t.uid
"""

views_diff = pd.read_sql_query(query, conn)
views_diff

,uid,avg_diff,pageviews
0,user_1,-65.119644,5
1,user_10,-75.242310,5
2,user_14,-159.568696,3
3,user_17,-62.207514,5
4,user_18,-6.367907,3
5,user_19,-99.440298,4
6,user_21,-96.111042,4
7,user_25,-93.474751,5
8,user_28,-86.793652,5
9,user_3,-105.738041,5


In [72]:
views_diff[['avg_diff', 'pageviews']].corr()

,avg_diff,pageviews
avg_diff,1.000000,0.118374
pageviews,0.118374,1.000000
